In [19]:
import os
import glob
import pandas as pd
import numpy as np
import pyproj
from pyproj import CRS, Transformer

In [20]:
files = glob.glob("data/*/dom1l-fp*.xyz")
#files = glob.glob("data/*/dom1l-aw*.xyz")

In [21]:
print(files[:3])

['data\\dom1l_05158026_Monheim_am_Rhein_EPSG25832_XYZ\\dom1l-fp_32349_5660_1_nw.xyz', 'data\\dom1l_05158026_Monheim_am_Rhein_EPSG25832_XYZ\\dom1l-fp_32349_5661_1_nw.xyz', 'data\\dom1l_05158026_Monheim_am_Rhein_EPSG25832_XYZ\\dom1l-fp_32349_5666_1_nw.xyz']


# Sample file conversion

In [22]:
fpath = 'data\\dom1l_05158026_Monheim_am_Rhein_EPSG25832_XYZ\\dom1l-fp_32349_5660_1_nw.xyz'

In [23]:
df = pd.read_csv(fpath, header=None, names=["x", "y", "z"])

In [24]:
df.head()

,x,y,z
0,349000.0,5660017.01,43.84
1,349000.0,5660019.82,50.58
2,349000.0,5660034.90,60.41
3,349000.0,5660057.71,43.85
4,349000.0,5660058.17,43.83


In [25]:
crs_4326 = CRS.from_epsg(4326) # WGS84
crs_25832 = CRS.from_epsg(25832) # UTM

In [26]:
crs_4326

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [27]:
crs_25832

<Projected CRS: EPSG:25832>
Name: ETRS89 / UTM zone 32N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe - 6°E to 12°E and ETRS89 by country
- bounds: (6.0, 38.76, 12.0, 83.92)
Coordinate Operation:
- name: UTM zone 32N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [28]:
to_wgs_84_transformer = Transformer.from_crs(crs_25832, crs_4326)

In [29]:
# test
print(to_wgs_84_transformer.transform(349643.89, 5660898.65))

(51.07987090140865, 6.853454480885797)


# All files

In [35]:
from tqdm import tqdm, tqdm_notebook
from multiprocessing import Pool, cpu_count

In [31]:
cores = cpu_count()

In [32]:
p = Pool(cores)

In [36]:
def calculate_wgs84(df_part):
    #df_part[["lat", "lon"]] = df_part.apply(lambda row: pd.Series(to_wgs_84_transformer.transform(row["x"], row["y"])), axis=1)
    lat_arr, lon_arr = list(), list()
    for i, (idx, row) in enumerate(df_part.iterrows()):
        res = to_wgs_84_transformer.transform(row["x"], row["y"])
        lat_arr.append(res[0]), lon_arr.append(res[0])
        
    return df_part

In [37]:
for f in files:
    df_file = pd.read_csv(f, header=None, names=["x", "y", "z"])
    
    print(f"{f}:{len(df_file)}")
    
    print("Calculating wgs84...")
    
    lat_arr, lon_arr = list(), list()
    for idx, row in tqdm_notebook(df_file.iterrows(), total=len(df_file)):
        res = to_wgs_84_transformer.transform(row["x"], row["y"])
        lat_arr.append(res[0]), lon_arr.append(res[1])

    df_file["lat"] = lat_arr
    df_file["lon"] = lon_arr
    
    df_file.to_csv(os.path.join("data/extracted", os.path.basename(f))[:-4]+".csv", index=False)

data\dom1l_05158026_Monheim_am_Rhein_EPSG25832_XYZ\dom1l-fp_32349_5660_1_nw.xyz:6077053
Calculating wgs84...


data\dom1l_05158026_Monheim_am_Rhein_EPSG25832_XYZ\dom1l-fp_32349_5661_1_nw.xyz:5161880
Calculating wgs84...


data\dom1l_05158026_Monheim_am_Rhein_EPSG25832_XYZ\dom1l-fp_32349_5666_1_nw.xyz:4694823
Calculating wgs84...


KeyboardInterrupt: 

In [38]:
df2 = pd.read_csv("data\extracted\dom1l-fp_32349_5660_1_nw.csv")

In [40]:
df2.head(500000).to_csv("data\extracted\dom1l-fp_32349_5660_1_nw_chunk1.csv")

In [41]:
df2.sample(500000).to_csv("data\extracted\dom1l-fp_32349_5660_1_nw_sample1.csv")